In [9]:
import xarray as xr
import matplotlib.pyplot as plt 
import fsspec
from datetime import datetime

In [ ]:
mapper = fsspec.get_mapper('s3://gstp-benchmark/zarr/s2-hypercube/s2_hypercube.zarr/', anon=True)
dataset = xr.open_zarr(mapper)
start = datetime.fromisoformat("2020-08-01")
end = datetime.fromisoformat("2020-08-31")
timeserie = dataset['Band1'].sel(Time=slice(start,end), Longitude=199995, Latitude=4499985).compute()
timeserie.plot.line(marker="o")

In [10]:
import numpy as np
import xarray as xr
import fsspec

# Open the Zarr dataset
mapper = fsspec.get_mapper('s3://gstp-benchmark/zarr/prisma/spain_20200804_modified.zarr/',anon=False)
dataset = xr.open_zarr(mapper)
#dataset = xr.open_zarr('E:\\test\\Satellite_images\\GSTP\\spain_20200804_modified.zarr', consolidated=True)

# Compute the list of desired indexes based on shape and iteration step
max_index = dataset.cube_wl_dim.size # 234 Expected  in this case
indexes = np.arange(0,max_index,10) # From 0 to Max with a step of 10
print(indexes) # [0  10  20  30  40  50  60  70  80  90 100 110 120 130 140 150 160 170 180 190 200 210 220 230]

# Condense the WaveLenght axis (with the list of index computed previously)
condensed = dataset['cube'].isel(cube_wl_dim=indexes)
print(condensed)

NoCredentialsError: Unable to locate credentials

In [11]:
import xarray as xr
import matplotlib.pyplot as plt 
from datetime import datetime

# Open the Zarr dataset
dataset = xr.open_zarr('E:\\test\\Satellite_images\\s2\\out.zarr', consolidated=True)

# Subset on the time axis
time = datetime.fromisoformat('2020-08-04T11:38:47').timestamp()
subset = dataset['Band1'].sel(Time=time).compute()

# Generate the plot from the subset (pseudocolor plot using predefined colormap 'terrain')
subset.plot.pcolormesh(cmap='terrain')

# Save figure to PNG file.
plt.savefig('E:\\test\\Satellite_images\\s2\\test_export_terrain.png')


KeyError: '.zmetadata'

In [ ]:
import xarray as xr

# Open the Zarr dataset.
dataset = xr.open_zarr('E:\\test\\Satellite_images\\GSTP\\s2_hypercube.zarr', consolidated=True)
# Display dataset metadata in the Standard output
print(dataset)
# Display only dataset coordinates
print("Coordinates: ",dataset.coords)
# Display only dataset attributes
print("Attributes: ",dataset.attrs)




In [12]:
import xarray as xr
import matplotlib.pyplot as plt 
from datetime import datetime

# Open the Zarr dataset
dataset = xr.open_zarr('E:\\test\\Satellite_images\\GSTP\\s2_hypercube.zarr', consolidated=True)

# Groub by month
b04 = dataset['Band3'].groupby("Time.month").mean("Time").isel(month=0)
print("Subset: ", b04)

b08 = dataset['Band4'].groupby("Time.month").mean("Time").isel(month=0)
print("Subset: ", b08)


ndvi = (b08 - b04) / (b08 + b04)

# Generate the plot from the subset (pseudocolor plot using predefined colormap 'terrain')
ndvi.plot.pcolormesh(cmap='terrain')

# Save figure to PNG file.
plt.savefig('E:\\test\\Satellite_images\\s2\\test_ndvi.png')


KeyError: '.zmetadata'

In [ ]:
# https://gis.stackexchange.com/questions/290138/extracting-values-at-specific-latitude-longitude-from-modis-swath-data
import xarray as xr
import numpy as np

dataset = xr.open_zarr('E:\\test\\Satellite_images\\GSTP\\prisma\\spain\\wavelength\\20200804111827.zarr')
print(dataset)

#Compute desired Lat/Lon indexes from SWATH structure
long_obs = -5.9 
lat_obs = 39.9 
Longitude =  dataset.longitude.data.compute()
Latitude =  dataset.latitude.data.compute()
coordinates = np.unravel_index((np.abs(Latitude - lat_obs) + np.abs(Longitude - long_obs)).argmin(), Latitude.shape)
print(coordinates)

#Apply subsetting
subset = dataset['cube'].sel(yc=coordinates[0], xc=coordinates[1], wavelength=slice(400,800)).compute()
print(subset)
#Apply mean function on the subsetted data.
result = np.mean(subset)
print('Mean: ', result.data)

In [ ]:
# Test reprojection from UTM30 to EPSG4326 with cartopy

import cartopy

src_crs = cartopy.crs.Geodetic() # Geodetic WGS84
dest_crs = cartopy.crs.UTM(30) # UTM Zone 30
points = src_crs.transform_point(199995,4499985,dest_crs) # Transform UTM Zone 30 coordinates to WGS 84
print(points)

import xarray as xr
from pyproj import Transformer

# Retrieve dataset and coordinates
dataset = xr.open_zarr('E:\\test\\Satellite_images\\s2\\out.zarr', consolidated=True)
lat = dataset.Latitude.data
lon = dataset.Longitude.data

# Reproject coordinates
transformer = Transformer.from_crs(32630, 4326) # UTM30 to EPSG 4326
latlon_reprojected = transformer.transform(lon,lat) #Reproject lat lon
new_lon = latlon_reprojected[0]
new_lat = latlon_reprojected[1]

# Assign reprojected coordinates
dataset = dataset.assign_coords(Latitude=new_lat)
dataset = dataset.assign_coords(Longitude=new_lon)
dataset = dataset.assign_coords({"Longitude": new_lon, "Latitude": new_lat})

# Save dataset
dataset.to_zarr('E:\\test\\Satellite_images\\s2\\out_reprojected_4326.zarr')


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt 
from datetime import datetime

# Open the Zarr dataset.
dataset = xr.open_zarr('E:\\test\\Satellite_images\\GSTP\\s2_hypercube.zarr', consolidated=True)

# Slice on the time axis of a specific geographial point.
start = datetime.fromisoformat("2020-08-01")
end = datetime.fromisoformat("2020-08-31")
timeserie = dataset['Band1'].sel(Time=slice(start,end), Longitude=199995, Latitude=4499985).compute()
print(timeserie)

# Generate the plot from the timeserie.
timeserie.plot.line(marker="o")

# Save figure to PNG file.
plt.savefig('E:\\test\\Satellite_images\\s2\\test_timeserie.png')



In [ ]:
import xarray as xr
import matplotlib.pyplot as plt 
from datetime import datetime

# Open the Zarr dataset
dataset = xr.open_zarr('E:\\test\\Satellite_images\\GSTP\\s2_hypercube.zarr', consolidated=True)

# Regroup on the time axis
mean = dataset['Band1'].groupby("Time.month").mean("Time")

print(mean)

subset = mean.isel(month=1)

print("Subset: ", subset)

# Generate the plot from the subset (pseudocolor plot using predefined colormap 'terrain')
subset.plot.pcolormesh(cmap='terrain')

# Save figure to PNG file.
plt.savefig('E:\\test\\Satellite_images\\s2\\test_time_mean.png')


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

# Open the Zarr dataset
dataset = xr.open_zarr('E:\\test\\Satellite_images\\s2\\out.zarr', consolidated=True)

# Resample on Longitude and Latitude by 4 times (mean function used for resampling)
resampled = dataset.coarsen(Longitude=4, Latitude=4).mean()

# Generate the plot from the subset (pseudocolor plot using predefined colormap 'terrain')
resampled['Band1'].isel(Time=0).plot.pcolormesh(cmap='terrain')

# Save figure to PNG file.
plt.savefig('E:\\test\\Satellite_images\\s2\\test_resampled.png')
